# CIA World Factbook Project
#### The CIA World Factbook is a collection of statistics on all of the countries on Earth.  For this project, we'll be using SQL to analyze some of the data from this particular database.  The columns that are in the database that we can analyze are as following:
- name : the name of the country
- area : the country's total area (both land and water)
- area_land : the country's land in square kilometers
- area_water : the country's water area in square kilometers
- population : the country's population
- population_growth : the country's population growth as a percentage
- birth_rate : the country's birth rate as the number of births per year per 1,000 people
- death_rate : the country's death rate as the number of deaths per year per 1,000 people

# Connect Jupyter Notebook to the database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

# Explore the database to see what tables are inside

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


# Facts is the table we're interested in
#### Let's begin running a query to bring all columns and data from the facts table.  Let's limit it to the first 5 rows.

In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Explore population statistics
#### We can look at the max and min of the poplulation and population growth

In [4]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop,
    MIN(population_growth) AS min_pop_growth,
    MAX(population_growth) AS max_pop_growth
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


#### It looks like we have countries with zero population and 7.2 billion population.  That doesn't make sense.  Let's explore the countries where this is the case.

In [5]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MIN(population) FROM facts); 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


#### The CIA Factbook page explains that Antartica does indeed of zero inhabitants, so we can leave that in.  However, since we're only interested in individual countries, we can remove the whole World row.

In [7]:
%%sql
SELECT MIN(population) AS min_pop, 
        MAX(population) AS max_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
FROM facts
WHERE name != ('World');

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


# Finding densely populated countries
#### We can determine these values by filtering based on countries that have above-average values for populationand have below-average values for area.

In [9]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population) 
                   FROM facts
                   WHERE name != 'World')
AND area < (SELECT AVG(area)
           FROM facts
           WHERE name != 'World');


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


# Country with most people

In [10]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MAX(population) FROM facts WHERE name != 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


# Country with the highest growth rate

In [11]:
%%sql
SELECT *
FROM facts
WHERE population_growth == (SELECT MAX(population_growth) FROM FACTS WHERE name != 'World')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


# Countries with the highest ratios of water to land

In [12]:
%%sql
SELECT *, CAST(area_water as Float) / CAST(area_land as Float) as water_to_land_ratio
FROM facts
ORDER BY water_to_land_ratio DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_to_land_ratio
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.6666666666666
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4.520231213872832
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.5547914317925592
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0,0.3866133866133866
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0,0.2846728307254623
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,0.25939625850340137
125,nl,Netherlands,41543,33893,7650,16947904,0.41,10.83,8.66,1.95,0.22571032366565366
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,0.22292237442922375
56,er,Eritrea,117600,101000,16600,6527689,2.25,30.0,7.52,0.0,0.16435643564356436
99,li,Liberia,111369,96320,15049,4195666,2.47,34.41,9.69,0.0,0.15623961794019933


# Countries that have more water than land

In [13]:
%%sql
SELECT *
FROM facts
WHERE area_water > area_land;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


# Countries that will most likely add the most people to their population next year

In [14]:
%%sql
SELECT *, population_growth
FROM facts
ORDER BY population_growth DESC;


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,population_growth_1
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47,4.02
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,3.32
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0,3.28
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56,3.25
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,3.24
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39,3.07
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0,3.03
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26,2.98
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None,2.95
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62,2.93


# Countries that have higher death rates than birth rates

In [15]:
%%sql
SELECT *
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6


# Countries with highest population/area ratio

In [16]:
%%sql
SELECT *, CAST(population as Float) / CAST(area as Float) AS pop_to_area_ratio
FROM facts
ORDER BY pop_to_area_ratio DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,pop_to_area_ratio
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168.964285714286
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267.5
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8141.279770444763
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6445.041516245487
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191.819444444444
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876.333333333333
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771.8592105263158
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319.6409395973155
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310.01582278481
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299.9259259259259
